In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import json


leagues_url = "https://oddspedia.com/api/v1/getLeagues?topLeaguesOnly=1&includeLeaguesWithoutMatches=1&sport=futebol&geoCode=BR&language=br"

driver = webdriver.Chrome()
driver.get(leagues_url)
leagues_list = json.loads(driver.find_element(By.TAG_NAME, 'pre').text)["data"]
driver.close()

In [2]:
br_leagues_url = "https://oddspedia.com/api/v1/getLeagues?topLeaguesOnly=0&includeLeaguesWithoutMatches=1&geoCode=BR&sport=futebol&category=brasil&startDate=2023-06-15T03%3A00%3A00Z&endDate=2023-06-16T02%3A59%3A59Z&language=br"

driver = webdriver.Chrome()
driver.get(br_leagues_url)
br_leagues_list = json.loads(driver.find_element(By.TAG_NAME, 'pre').text)["data"]
driver.close()

In [3]:
br_leagues_list_minimal = []
br_names = set()
for league in br_leagues_list:
    if "sub" in league["league_name"].lower() or "feminino" in league["league_name"].lower() or "u20" in league["league_name"].lower():
        continue

    if len(league["league_name"].split()[-1]) in [1, 2]:
        if league["league_name"].split()[-1].upper() in ["A", "1", "A1"]:
            br_leagues_list_minimal.append(league.copy())
            br_names.add(league["league_name"])
    else:
        br_leagues_list_minimal.append(league.copy())

In [4]:
from operator import itemgetter

keep = []
for i, league in enumerate(leagues_list):
    if league["league_name"] not in br_names:
        keep.append(i)
    
leagues_list = list(itemgetter(*keep)(leagues_list))
for league in br_leagues_list_minimal:
    leagues_list.append(league)

In [5]:
from datetime import datetime, timedelta
import re


def parse_time(dm, year, delta):
    if re.match(string=dm, pattern="\d{2}:\d{2}"):
        dm = dm[5:]
    if re.match(string=dm, pattern="\d{4}"):
        dm = dm[:-5]
    dm = dm.strip()
    
    month_to_number = {
        'janeiro': 1,
        'fevereiro': 2,
        'março': 3,
        'abril': 4,
        'maio': 5,
        'junho': 6,
        'julho': 7,
        'agosto': 8,
        'setembro': 9,
        'outubro': 10,
        'novembro': 11,
        'dezembro': 12
    }
    day, month = dm.split(" de ")[:2]
    month = month_to_number[month]
    
    corrected_dt = datetime.strptime(f"{year}-{month}-{day.strip()}", "%Y-%m-%d")
    
    corrected_dt += delta

    return (corrected_dt).strftime("%Y-%m-%d")

In [12]:
cancelled = {}
out = {}

In [13]:
import requests
from bs4 import BeautifulSoup


YEARS = range(2019, 2023)
GOOGLE = "https://www.google.com"
for i, league in enumerate(leagues_list):
    out[league['league_name']] = {
        "category": league["category_slug"],
        "alias": league["league_slug"]
    }
    if league['league_name'] == "Eurocopa":
        continue
    for year in YEARS:
        print(league['league_name'], i, year)
        
        response = requests.get(f"{GOOGLE}/search?q={league['league_name'].replace(' ', '+').replace('ã', 'a')}+{year}+periodo")
        soup = BeautifulSoup(response.text)
        
        try:
            dates_raw = soup.find("div", {"class": "BNeawe iBp4i AP7Wnd"}).text
        except AttributeError:
            if league['league_name'] not in cancelled:
                cancelled[league['league_name']] = {}
            cancelled[league['league_name']][year] = "Requer analise manual"
            continue

        if dates_raw == "Cancelado":
            if league['league_name'] not in cancelled:
                cancelled[league['league_name']] = {}
            cancelled[league['league_name']][year] = "Cancelado"
            continue
        
        out[league['league_name']][year] = {}
        if league['league_name'] == "Supercopa do Brasil":
            date = dates_raw.split(", ")[1]
            start_date = date
            end_date = date
        elif league["league_name"] == "Copa Verde" and year < 2022:
            start_date, end_date = dates_raw.split(" – ")
            start_date = start_date[:-8]
            end_date = end_date[:-8]
        else:
            start_date, end_date = dates_raw.split("De: ")[1].split(" Até: ")
        
        out[league['league_name']][year]["start_date"] = parse_time(start_date, year, -timedelta(days=7))
        out[league['league_name']][year]["end_date"] = parse_time(end_date, year, timedelta(days=7))

Copa América 0 2019
Copa América 0 2020
Copa América 0 2021
Copa América 0 2022
Copa do Brasil 1 2019
Copa do Brasil 1 2020
Copa do Brasil 1 2021
Copa do Brasil 1 2022
CONMEBOL Libertadores 2 2019
CONMEBOL Libertadores 2 2020
CONMEBOL Libertadores 2 2021
CONMEBOL Libertadores 2 2022
Copa Sul-Americana 4 2019
Copa Sul-Americana 4 2020
Copa Sul-Americana 4 2021
Copa Sul-Americana 4 2022
Liga dos Campeões 5 2019
Liga dos Campeões 5 2020
Liga dos Campeões 5 2021
Liga dos Campeões 5 2022
Liga Europa 6 2019
Liga Europa 6 2020
Liga Europa 6 2021
Liga Europa 6 2022
Premier League 7 2019
Premier League 7 2020
Premier League 7 2021
Premier League 7 2022
Bundesliga 8 2019
Bundesliga 8 2020
Bundesliga 8 2021
Bundesliga 8 2022
Primeira Divisão 9 2019
Primeira Divisão 9 2020
Primeira Divisão 9 2021
Primeira Divisão 9 2022
Série A 10 2019
Série A 10 2020
Série A 10 2021
Série A 10 2022
Ligue 1 11 2019
Ligue 1 11 2020
Ligue 1 11 2021
Ligue 1 11 2022
Liga Conferência 12 2019
Liga Conferência 12 2020
Li

In [14]:
import json

with open("base_info/collected.json", "w+") as file:
    json.dump(out, file)

with open("base_info/manual_collection.json", "w+") as file:
    json.dump(cancelled, file)